In [3]:
import re
import requests
from bs4 import BeautifulSoup

In [5]:
wiki_toronto = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content, 'lxml')

In [8]:
for item in wiki_toronto.table.find_all('td'):
    postal_code = item.p.b.text
    if item.p.span.text == "Not assigned":
        continue
    match = re.match("(?P<borough>.+)(\s\((?P<neighborhood>.+)\))*", item.p.span.text)
    
    if match is None:
        print(item.p.span.text)
        break
        
    print("{0} - {1}".format(match.group('borough'), match.group('neighborhood')))

North York - Parkwoods
North York - Victoria Village
Downtown Toronto - Regent Park / Harbourfront
North York - Lawrence Manor / Lawrence Heights
Queen's Park - None
Etobicoke - Islington Avenue
Scarborough - Malvern / Rouge
North York - Don Mills
East York - Parkview Hill / Woodbine Gardens
Downtown Toronto - Garden District, Ryerson
North York - Glencairn
Etobicoke - West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Scarborough - Rouge Hill / Port Union / Highland Creek
North York - Don Mills
East York - Woodbine Heights
Downtown Toronto - St. James Town
York - Humewood-Cedarvale
Etobicoke - Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Scarborough - Guildwood / Morningside / West Hill
East Toronto - The Beaches
Downtown Toronto - Berczy Park
York - Caledonia-Fairbanks
Scarborough - Woburn
East York - Leaside
Downtown Toronto - Central Bay Street
Downtown Toronto - Christie
Scarborough - Cedarbrae
North York - Hillcrest Village
North York -